<a href="https://colab.research.google.com/github/Spin-Chemistry-Labs/radicalpy/blob/187-google-colab-tutorials/examples/tutorials/05_introduction_to_magnetic_field_effects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Tutorial 5 - Incoherent processes


&copy; Lewis M. Antill, 2025

This tutorial aims to...

In [ ]:
!pip install radicalpy

In [38]:
import matplotlib.pyplot as plt
import numpy as np

import radicalpy as rp
from radicalpy.classical import Rate, RateEquations
from radicalpy.data import Triplet
from radicalpy.experiments import anisotropy, mary, kine_quantum_mary, steady_state_mary
from radicalpy.kinetics import Haberkorn, HaberkornFree
from radicalpy.plot import plot_general
from radicalpy import relaxation
from radicalpy.simulation import Basis, Molecule, LiouvilleSimulation, SemiclassicalSimulation, State
from radicalpy.utils import Bhalf_fit